In [1]:
import os, sys, time, random
import h5py
import numpy as np
print(os.listdir("C:/DEVEL/code/data/"))

['annotation', 'inventory.csv', 'lstm_version_trainset.csv', 'ner_trainsets', 'old', 'v_GS_ADD_REMOVE_PROGRAMS.txt']


In [2]:
from keras.layers import *
from keras.models import *
from keras.utils import *
from keras.initializers import *
import tensorflow as tf

In [3]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 128  # Latent dimensionality of the encoding space.
num_samples = 8000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = "C:/DEVEL/code/data/lstm_version_trainset.csv"

In [4]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, extra_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [5]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 8000
Number of unique input tokens: 76
Number of unique output tokens: 38
Max sequence length for inputs: 182
Max sequence length for outputs: 21


In [6]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.


In [7]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [8]:
model.summary()
# print(model.layers[-1].input)
print(model.layers[-3].output)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 76)]   0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None, 38)]   0           []                               
                                                                                                  
 lstm (LSTM)                    [(None, 128),        104960      ['input_1[0][0]']                
                                 (None, 128),                                                     
                                 (None, 128)]                                                     
                                                                                              

In [9]:
print("encoder_input_data shape:",encoder_input_data.shape)
print("decoder_input_data shape:",decoder_input_data.shape)
print("decoder_target_data shape:",decoder_target_data.shape)

encoder_input_data shape: (8000, 182, 76)
decoder_input_data shape: (8000, 21, 38)
decoder_target_data shape: (8000, 21, 38)


In [10]:
plot_model(model,show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [11]:

# Run training
from keras.optimizers import * 
model.compile(optimizer=Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.001), loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs, 
          validation_split=0.2)

c:\Users\humbe\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/100
100/100 [==============================] - 6s 31ms/step - loss: 0.6742 - val_loss: 0.6890
Epoch 2/100
100/100 [==============================] - 2s 24ms/step - loss: 0.6324 - val_loss: 0.6829
Epoch 3/100
100/100 [==============================] - 2s 24ms/step - loss: 0.6273 - val_loss: 0.6963
Epoch 4/100
100/100 [==============================] - 2s 23ms/step - loss: 0.6852 - val_loss: 0.6876
Epoch 5/100
100/100 [==============================] - 2s 24ms/step - loss: 0.6241 - val_loss: 0.6824
Epoch 6/100
100/100 [==============================] - 2s 24ms/step - loss: 0.6154 - val_loss: 0.6752
Epoch 7/100
100/100 [==============================] - 2s 23ms/step - loss: 0.6112 - val_loss: 0.6743
Epoch 8/100
100/100 [==============================] - 2s 23ms/step - loss: 0.6083 - val_loss: 0.6720
Epoch 9/100
100/100 [==============================] - 2s 23ms/step - loss: 0.6065 - val_loss: 0.6692
Epoch 10/100
100/100 [==============================] - 2s 24ms/step - loss: 0.604

In [12]:
# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [13]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [14]:

for seq_index in range(10):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

1/1 [==============================] - 0s 17ms/step
-
Input sentence: title
Decoded sentence: 1.0...................
1/1 [==============================] - 0s 18ms/step
-
Input sentence: 01org Tpm2.0-tools 1.1.0
Decoded sentence: 1.0...................
1/1 [==============================] - 0s 18ms/step
-
Input sentence: 0xacab mat2
Decoded sentence: 1.0...................
1/1 [==============================] - 0s 18ms/step
-
Input sentence: 0xacab mat2 0.1.0
Decoded sentence: 1.0...................
1/1 [==============================] - 0s 18ms/step
-
Input sentence: 0xacab mat2 0.1.1
Decoded sentence: 1.0...................
1/1 [==============================] - 0s 18ms/step
-
Input sentence: 0xacab mat2 0.1.2
Decoded sentence: 1.0...................
1/1 [==============================] - 0s 18ms/step
-
Input sentence: 0xacab mat2 0.1.3
Decoded sentence: 1.0...................
1/1 [==============================] - 0s 17ms/step
-
Input sentence: 0xacab mat2 0.2.0
Decoded sentence: 1.